# PReLU — Matematiksel Temel, Türev ve Backprop Yorumu

Bu defter, **PReLU (Parametric ReLU)** aktivasyonunun matematiğini **okunur ve neden–sonuç odaklı** biçimde ele alır.

Kapsam:
- Parçalı tanım
- Süreklilik ve türev
- Backprop’ta gradyan akışı
- Öğrenilebilir parametrenin (a) etkisi
- ReLU / LeakyReLU ile matematiksel fark
- Pratik çıkarımlar

Amaç: Formül ezberi değil, **“buradan dolayı bu oluyor”** ilişkisini kurmak.

## 1) Tanım

PReLU, ReLU ailesine ait **parçalı doğrusal** bir aktivasyon fonksiyonudur.

\[
\mathrm{PReLU}(x) =
\begin{cases}
x, & x \ge 0 \\
a \, x, & x < 0
\end{cases}
\]

Burada:
- \(x\): ön-aktivasyon (pre-activation)
- \(a\): **öğrenilebilir** negatif eğim parametresi

Önemli fark:
- ReLU: \(a = 0\)
- LeakyReLU: \(a = \text{sabit}\)
- **PReLU:** \(a\) eğitim sırasında **öğrenilir**

## 2) Süreklilik

Fonksiyon her noktada süreklidir.

Özellikle \(x = 0\) noktasında:

- Sol limit: \(\lim_{x \to 0^-} a x = 0\)
- Sağ limit: \(\lim_{x \to 0^+} x = 0\)

İki parça aynı değeri verdiği için PReLU,
\[
x = 0 \text{ noktasında süreklidir.}
\]

## 3) Türev (x ≠ 0)

PReLU’nun türevi, parça parça şu şekildedir:

\[
\frac{d}{dx}\mathrm{PReLU}(x) =
\begin{cases}
1, & x > 0 \\
a, & x < 0
\end{cases}
\]

### x = 0 noktasında

- Sol türev: \(a\)
- Sağ türev: \(1\)

Bu nedenle klasik anlamda tek bir türev yoktur.  
Derin öğrenmede bu nokta **ölçüsü sıfır** olan bir küme olduğu için **subgradient** yaklaşımı kullanılır.

## 4) Backprop: gradyan neden kaybolmaz?

Bir katmanda:
\[
z = Wx + b, \quad y = \mathrm{PReLU}(z)
\]

Kayıp fonksiyonu:
\[
\mathcal{L} = \mathcal{L}(y)
\]

Zincir kuralı:
\[
\frac{\partial \mathcal{L}}{\partial z}
=
\frac{\partial \mathcal{L}}{\partial y}
\cdot
\frac{dy}{dz}
=
\delta \cdot \mathrm{PReLU}'(z)
\]

Burada:
\[
\delta = \frac{\partial \mathcal{L}}{\partial y}
\]

### Negatif ve pozitif bölgede gradyan

- Eğer \(z > 0\):
\[
\frac{\partial \mathcal{L}}{\partial z} = \delta
\]

- Eğer \(z < 0\):
\[
\frac{\partial \mathcal{L}}{\partial z} = a \, \delta
\]

### Buradan dolayı:

- ReLU’da (\(a=0\)) negatif bölgede gradyan **tamamen sıfırlanır**
- LeakyReLU’da gradyan **sabit bir katsayıyla** akar
- **PReLU’da gradyan hem akar hem de büyüklüğü öğrenilebilir**

## 5) Öğrenilebilir parametre a için gradyan

PReLU’nun kritik farkı:  
Kayıp fonksiyonu **a parametresine de bağlıdır**.

Negatif bölgede (\(z<0\)):

\[
y = a z
\]

Bu durumda:
\[
\frac{\partial \mathcal{L}}{\partial a}
=
\frac{\partial \mathcal{L}}{\partial y}
\cdot
\frac{\partial y}{\partial a}
=
\delta \cdot z
\]

### Yorum

- Eğer \(z < 0\) ve \(\delta\) büyükse → \(a\) güçlü şekilde güncellenir
- Model, negatif tarafta **ne kadar bilgi geçirmek gerektiğini** veriden öğrenir

Bu, LeakyReLU’da **olmayan** bir serbestliktir.

## 6) Kanal başına PReLU (Channel-wise) matematiği

Eğer her kanal için ayrı \(a_c\) parametresi kullanılırsa:

\[
y_c =
\begin{cases}
z_c, & z_c \ge 0 \\
a_c z_c, & z_c < 0
\end{cases}
\]

Her kanalın gradyanı **kendi dağılımına göre** şekillenir.

### Sonuç:
- Daha yüksek temsil gücü
- Ama daha fazla parametre ve overfitting riski

## 7) Ölçek ve optimizasyon yorumu

Negatif tarafta eğimin öğrenilebilir olması şu anlama gelir:

- Model, bazı katmanlarda ReLU’ya yakın davranabilir (\(a \to 0\))
- Bazı katmanlarda daha lineer davranabilir (\(a\) büyür)

Bu durum:
- Optimizasyon sürecini daha **esnek**
- Ama aynı zamanda daha **hassas**
hale getirir.

## 8) ReLU – LeakyReLU – PReLU (matematiksel özet)

| Aktivasyon | Negatif eğim | Öğrenilebilir mi | Gradyan |
|-----------|--------------|------------------|---------|
| ReLU | 0 | ❌ | Negatifte 0 |
| LeakyReLU | Sabit \(a\) | ❌ | Negatifte \(a\delta\) |
| PReLU | \(a\) | ✅ | Negatifte öğrenilen \(a\delta\) |

PReLU, matematiksel olarak **en esnek**, ama en riskli seçenektir.

## 9) Sonuç (matematikten çıkan pratik)

Matematiksel analiz şunu gösterir:

- PReLU, ReLU’nun gradyan kesilmesi problemini çözer
- LeakyReLU’dan farklı olarak bu çözümü **veriye adapte eder**
- Bu esneklik:
  - derin ve karmaşık modellerde avantaj
  - küçük veri / kısa eğitimde dezavantaj
olabilir

Bu yüzden PReLU:
- backbone güçlendirme
- research / ablation
senaryolarında matematiksel olarak anlamlıdır.